In [ ]:

!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7



In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import pandas as pd
from datasets import load_dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import json
import re
from pprint import pprint
import transformers
from transformers.utils import send_example_telemetry
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import torch
import numpy as np

from pathlib import Path
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from sklearn.model_selection import train_test_split
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
with Path("/content/drive/MyDrive/Grad/BioASQ-train-factoid-4b.json").open() as json_file:

    dataset = json.load(json_file)
    data=dataset["data"][0]["paragraphs"]

drs = pd.json_normalize(data)
da = Dataset.from_pandas(drs)
da




Dataset({
    features: ['qas', 'context'],
    num_rows: 3266
})

In [ ]:
def extract_questions_and_answers(factoid_path: Path):
    with factoid_path.open() as json_file:
        data = json.load(json_file)

    questions = data["data"][0]["paragraphs"]

    data_rows = []


    for question in questions:
        context = question["context"]
        for question_and_answers in question["qas"]:
            question = question_and_answers["question"]
            answers = question_and_answers["answers"]
            for answer in answers:
                answer_text = answer["text"]
                answer_start = answer["answer_start"]
                answer_end = answer_start + len(answer_text)

                data_rows.append({
                    "question": question,
                    "context": context,
                    "answer_text": answer_text,
                    "answer_start": answer_start,
                    "answer_end": answer_end
                })
    return pd.DataFrame(data_rows)

In [ ]:
factoid_paths = [sorted(list(Path("/content/drive/MyDrive/Grad/").glob("BioASQ-train-*")))]
filtered_paths = [path for path in factoid_paths[0] if '4b' in str(path) or '5b' in str(path)]
filtered_paths

[PosixPath('/content/drive/MyDrive/Grad/BioASQ-train-factoid-4b.json'),
 PosixPath('/content/drive/MyDrive/Grad/BioASQ-train-factoid-5b.json')]

In [ ]:
dfs = []

for factoid_path in filtered_paths:
    dfs.append(extract_questions_and_answers(factoid_path))


df = pd.concat(dfs)

In [ ]:
df.head()

,question,context,answer_text,answer_start,answer_end
0,What is the inheritance pattern of Li–Fraumeni...,Balanced t(11;15)(q23;q15) in a TP53+/+ breast...,autosomal dominant,213,231
1,What is the inheritance pattern of Li–Fraumeni...,Genetic modeling of Li-Fraumeni syndrome in ze...,autosomal dominant,105,123
2,Which type of lung cancer is afatinib used for?,Clinical perspective of afatinib in non-small ...,EGFR-mutant NSCLC,1203,1220
3,Which hormone abnormalities are characteristic...,"DOCA sensitive pendrin expression in kidney, h...",thyroid,419,426
4,Which hormone abnormalities are characteristic...,Clinical and molecular characteristics of Pend...,thyroid,705,712


In [ ]:
# df = df.drop_duplicates(subset = ["context"]).reset_index(drop = True)
df = df.drop_duplicates()
df

,question,context,answer_text,answer_start,answer_end
0,What is the inheritance pattern of Li–Fraumeni...,Balanced t(11;15)(q23;q15) in a TP53+/+ breast...,autosomal dominant,213,231
1,What is the inheritance pattern of Li–Fraumeni...,Genetic modeling of Li-Fraumeni syndrome in ze...,autosomal dominant,105,123
2,Which type of lung cancer is afatinib used for?,Clinical perspective of afatinib in non-small ...,EGFR-mutant NSCLC,1203,1220
3,Which hormone abnormalities are characteristic...,"DOCA sensitive pendrin expression in kidney, h...",thyroid,419,426
4,Which hormone abnormalities are characteristic...,Clinical and molecular characteristics of Pend...,thyroid,705,712
...,...,...,...,...,...
4923,Which enzyme is inhibited by Imetelstat?,The telomerase inhibitor imetelstat depletes c...,telomerase,639,649
4925,Which enzyme is inhibited by Imetelstat?,Telomerase Inhibitor Imetelstat in Patients wi...,telomerase,199,209
4934,Which enzyme is inhibited by Imetelstat?,A Pilot Study of the Telomerase Inhibitor Imet...,telomerase,338,348
4939,Which enzyme is inhibited by Imetelstat?,"The telomerase inhibitor imetelstat alone, and...",Telomerase,440,450


In [ ]:

# # Convert the Dataset to a pandas DataFrame
# df = da.to_pandas()

# # Convert NumPy arrays to strings
# df = df.applymap(lambda x: ', '.join(map(str, x)) if isinstance(x, np.ndarray) else x)

# # Identify and remove duplicate rows
# duplicate_mask = df.duplicated()
# df_flattened = df[~duplicate_mask]

# Convert the pandas DataFrame back to a Dataset
da = Dataset.from_pandas(df)

# Split the data into training, testing, and validation sets
train_data, test_val_data = train_test_split(da, test_size=0.3, random_state=42)

# Ensure that test_val_data is a Dataset
if isinstance(test_val_data, dict):
    # Convert to pandas DataFrame
    test_val_data = pd.DataFrame.from_dict(test_val_data)
    # Convert to Dataset
    test_val_data = Dataset.from_pandas(test_val_data)
elif not isinstance(test_val_data, Dataset):
    test_val_data = Dataset.from_pandas(test_val_data)

# Reset the index to avoid indexing issues during the second split
test_val_data = test_val_data.to_pandas().reset_index(drop=True)

# Split the combined test_val_data into separate testing and validation sets
test_data, val_data = train_test_split(test_val_data, test_size=0.33, random_state=42)

train_data

{'question': ['What is the target of the drug Olaparib?',
  'Which hormone abnormalities are common in Williams syndrome ?',
  'Is the long non- coding RNA malat-1 up or downregulated in cancer?',
  'Is the transcriptional regulator BACH1 an activator or a repressor?',
  'What is the typical rash associated with gluten ?',
  'Which bone protein is used in archaelogy for dating and species identification?',
  'Which hormone deficiency is implicated in the Costello syndrome ?',
  "What is the mode of inheritance of Wilson's disease?",
  'Which is the gene mutated in type 1 neurofibromatosis?',
  'Which enzyme is inhibited by Imetelstat?',
  'Which is the RNA sequence of the canonical polyadenylation signal?',
  'Which is the most widely used model for the study of multiple sclerosis (MS)?',
  'Which protein has been found to interact with phospholamban (PLN) and is also an anti-apoptotic protein?',
  'Which protein is causing Netherton syndrome?',
  'Which metabolite activates AtxA?',
  

In [ ]:
# Assuming train_data is a dictionary
train_df = pd.DataFrame.from_dict(train_data)
# rms = pd.json_normalize(train_df)
train = Dataset.from_pandas(train_df)
train_df = pd.DataFrame.from_dict(train_data)
validate = Dataset.from_pandas(val_data)
validate

KeyError: ignored

In [ ]:
# Split the data into train (60%) and the rest (40%)
train_df, test_val_df = df.sample(frac=0.6, random_state=42), df.drop(train_df.index)

# Further split the test_val set into test (20%) and validation (20%)
test_df, val_df = test_val_df.sample(frac=0.50, random_state=42), test_val_df.drop(test_df.index)

# Save the dataframes to .jsonl files
train_df.to_json('train.jsonl', orient='records', lines=True)
test_df.to_json('test.jsonl', orient='records', lines=True)
val_df.to_json('val.jsonl', orient='records', lines=True)


In [ ]:
test_df

,question,context,answer_text,answer_start,answer_end
3059,What is the gene frequently mutated in Multipl...,Multiple endocrine neoplasia type 2 RET protoo...,RET,397,400
2457,Which JAK (Janus kinase) inhibitor is approved...,The JAK inhibitor tofacitinib regulates synovi...,Tofacitinib,149,160
963,Which MAP kinase phosphorylates the transcript...,A novel compound RY10-4 induces apoptosis and ...,JNK,1114,1117
3410,Which gene has been implicated in Majeed Syndr...,Efficacy of anti-IL-1 treatment in Majeed synd...,LPIN2,301,306
1868,Which is the target of the drug Denosumab?,Update on denosumab in the management of postm...,receptor activator of nuclear factor-κB ligand,636,682
...,...,...,...,...,...
141,What is the target of the drug Olaparib?,Phase II study of olaparib in patients with re...,PARP,379,383
139,What is the target of the drug Olaparib?,"BRCA1, PARP1 and γH2AX in acute myeloid leukem...",PARP,219,223
1724,"Which tumor suppressor is referred to as ""the ...",p53-Based cyclotherapy: exploiting the 'guardi...,p53,0,3
702,What disease is small bowel lymphoma commonly ...,Mortality in patients with celiac disease. Cel...,Celiac disease,43,57


In [ ]:
val_df

,question,context,answer_text,answer_start,answer_end
36,Which hormone abnormalities are characteristic...,Genetic factors that might lead to different r...,thyroid,1092,1099
38,Which hormone abnormalities are characteristic...,Genetics of congenital hypothyroidism. Congeni...,thyroid,877,884
60,What is the major adverse effect of adriamycin...,Pharmacokinetic-pharmacodynamic relationships ...,Cardiotoxicity,729,743
65,Which is the branch site consensus sequence in...,Mutational analysis of the U12-dependent branc...,UUCCUUAAC,464,473
86,Which is the main regulatory molecule of SERCA...,SERCA2a superinhibition by human phospholamban...,Phospholamban,110,123
...,...,...,...,...,...
4658,Does dasatinib promote or inhibit T-cell proli...,Dasatinib inhibits the proliferation and funct...,inhibits,10,18
4672,Which protein is found to be mutated in Friedr...,The iron-binding CyaY and IscX proteins assist...,frataxin,117,125
4731,Which type of cells is affected in Amyotrophic...,Amyotrophic lateral sclerosis model derived fr...,motor neurons,1032,1045
4892,Which enzyme is inhibited by Imetelstat?,Effect of telomerase inhibition on preclinical...,telomerase,757,767


In [ ]:
# Load datasets
train_dataset = load_dataset('json', data_files='/content/train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='/content/test.jsonl', split="train")

# Preprocess datasets
train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <>\n{system_message.strip()}\n<>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [f'[INST] <>\n{system_message.strip()}\n<>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,  # Pass validation dataset here
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

# Cell 4: Test the model
logging.set_verbosity(logging.CRITICAL)
prompt = f"[INST] <>\n{system_message}\n<>\n\nWrite a function that reverses a string. [/INST]" # replace the command here with something relevant to your task
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(prompt)
print(result[0]['generated_text'])


In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
#dataset_name = "derek-thomas/squad-v1.1-t5-question-generation"

# Fine-tuned model name
new_model = "llama-2-7b-miniguanaco"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 2

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 1

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
import matplotlib.pyplot as plt

# Load dataset (you can process it here)
# dataset = load_dataset(drs, split="train")
# train_dataloader = DataLoader(train, batch_size=per_device_train_batch_size, shuffle=True)
# val_dataloader = DataLoader(validate, batch_size=per_device_train_batch_size, shuffle=False)


# train_df = drs.sample(frac=0.8, random_state=42)
# val_df = drs.drop(train_df.index)
# ta = Dataset.from_pandas(train_df)

# va = Dataset.from_pandas(val_df)

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
)





config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train,
    eval_dataset=validate,
    peft_config=peft_config,
    dataset_text_field="context",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
train_results = trainer.train()
train_loss = train_results.training_loss

# Save trained model
trainer.model.save_pretrained(new_model)



In [ ]:

# Plotting training loss
plt.plot(train_loss, label="Training Loss")
plt.xlabel("Steps")
plt.ylabel("Loss")
plt.title("Training Loss Over Steps")
plt.legend()
plt.show()

In [ ]:
# prompt: after training test the dataset and generate predicted labels to calulate Recall, Accuracy, Precision, and F1


# Evaluate model on validation set
eval_results = trainer.evaluate()
print(eval_results)

# Calculate metrics
recall = eval_results["context"]
accuracy = eval_results["eval_accuracy"]
precision = eval_results["eval_precision"]
f1 = eval_results["eval_f1"]

# Print metrics
print("Recall:", recall)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1:", f1)


{'eval_loss': 1.1788383722305298, 'eval_runtime': 123.4283, 'eval_samples_per_second': 5.291, 'eval_steps_per_second': 0.664, 'epoch': 2.0}


KeyError: ignored

In [ ]:
%tensorboard --logdir ./lightning_logs

UsageError: Line magic function `%tensorboard` not found.


In [ ]:

# %load_ext tensorboard
# %tensorboard --logdir results/runs
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)
the_new ="Youssef115/llama-2-7b-miniguanaco"
# Run text generation pipeline with our next model
prompt = "Hi w"
pipe = pipeline(task="text-generation", model=the_new, tokenizer=tokenizer, max_length=100)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:

# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:

!huggingface-cli login

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Assuming 'drs' is defined elsewhere
# ...

# Assuming 'per_device_train_batch_size' is defined
# ...

# Load the trained model and tokenizer
model_name = "Youssef115/llama-2-7b-miniguanaco"  # Replace with the actual model name
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create a test dataset (assuming you have a test split)
test_df = drs.sample(frac=0.2, random_state=42)  # Adjust the fraction as needed
test_dataset = Dataset.from_pandas(test_data)

# Define a DataLoader for the test dataset
test_dataloader = DataLoader(test_dataset, batch_size=per_device_train_batch_size, shuffle=False)

# Set the model to evaluation mode
model.eval()

# Lists to store generated text and original prompts
generated_texts = []
original_prompts = []

# Iterate through the test dataset
for batch in test_dataloader:
    # Extract prompts from the batch
    prompts = batch["question"]  # Replace with the actual column name
    original_prompts.extend(prompts)

    # Generate text using the model
    inputs = tokenizer(prompts, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)

    # Decode the generated output
    generated_texts.extend(tokenizer.batch_decode(outputs["logits"], skip_special_tokens=True))

# Print a few generated texts and corresponding prompts
for prompt, generated_text in zip(original_prompts[:5], generated_texts[:5]):
    print(f"Prompt: {prompt}")
    print(f"Generated Text: {generated_text}")
    print("=" * 80)

# Optionally, plot the generated text length distribution
plt.hist([len(text.split()) for text in generated_texts], bins=30, color='blue', alpha=0.7)
plt.title('Generated Text Length Distribution')
plt.xlabel('Length of Generated Text')
plt.ylabel('Frequency')
plt.show()


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import DataLoader
from datasets import Dataset
import torch

# Load the fine-tuned model and tokenizer
model_name = "Youssef115/llama-2-7b-miniguanaco" # Replace with your actual model name
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Assuming you have a test dataset with true labels
# test_dataset = ...

# Create a DataLoader for the test dataset
test_dataloader = DataLoader(test_dataset, batch_size=per_device_train_batch_size, shuffle=False)

# Set the model to evaluation mode
model.eval()

# Lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Iterate through the test dataset
for batch in test_dataloader:
    prompts = batch["question"]  # Replace with your actual column name
    true_labels.extend(batch["true_labels"])  # Replace with your actual column name

    # Generate text using the model
    inputs = tokenizer(prompts, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)

    # Decode the generated output
    generated_texts = tokenizer.batch_decode(outputs["logits"], skip_special_tokens=True)
    predicted_labels.extend(generated_texts)

# Convert labels to appropriate format (e.g., convert strings to integers)
true_labels = [label_to_int(label) for label in true_labels]
predicted_labels = [label_to_int(label) for label in predicted_labels]

# Calculate metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average="weighted")
recall = recall_score(true_labels, predicted_labels, average="weighted")
f1 = f1_score(true_labels, predicted_labels, average="weighted")

# Print or use the calculated metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]